In [1]:
import cv2
import numpy as np
import os

from keras.models import Sequential
from keras.layers import Activation, Dense, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [0]:
img_width, img_height = 300//4, 200//4

In [0]:
paperDir = 'rockpaperscissors/rockpaperscissors/paper/'
rockDir = 'rockpaperscissors/rockpaperscissors/rock/'
scissorDir = 'rockpaperscissors/rockpaperscissors/scissors/'

paperList = os.listdir(paperDir)
rockList = os.listdir(rockDir)
scissorList = os.listdir(scissorDir)

dirTrain = [paperList[:500], rockList[:500], scissorList[:500]]
dirValidation = [paperList[500:650], rockList[500:650], scissorList[500:650]]

In [0]:
def preprocessing(img_name = None, i = None):
    # print(img_name, i)
    if i == 0:
        img_dir = paperDir
    elif i == 1:
        img_dir = rockDir
    else:
        img_dir = scissorDir

    img = cv2.imread(img_dir+img_name) # Open image

    min_HSV = np.array([0, 60, 40], dtype = "uint8")
    max_HSV = np.array([33, 255, 255], dtype = "uint8")
    hsvImg = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Skin color detection
    binaryImg = cv2.inRange(hsvImg, min_HSV, max_HSV)
    masked = cv2.bitwise_and(img, img, mask=binaryImg)
    # Mask the skin only 
    result = cv2.cvtColor(masked, cv2.COLOR_BGR2GRAY)
    # resize to desired size
    result = cv2.resize(result, (img_width, img_height))
    # expand to fit with keras
    result = np.expand_dims(result, axis=2)
    # print(binaryImg)
    # cv2.imshow('See This', result)
    # cv2.waitKey(0)

    return result

In [0]:
iTrain = 0
fileTrain = []
indexTrain = []

for dir in dirTrain:
    for file in dir:
        # print(file, iTrain)
        newImage = preprocessing(file, iTrain)
        fileTrain.append(newImage)
        if iTrain == 0:
            indexTrain.append([1,0,0])
        elif iTrain == 1:
            indexTrain.append([0,1,0])
        elif iTrain == 2:
            indexTrain.append([0, 0, 1])

    iTrain+=1

In [0]:
iVal = 0
fileValidation = []
indexValidation = []

for dir in dirValidation:
    for file in dir:
        newImage = preprocessing(file, iVal)
        fileValidation.append(newImage)
        if iVal == 0:
            indexValidation.append([1,0,0])
        elif iVal == 1:
            indexValidation.append([0,1,0])
        elif iVal == 2:
            indexValidation.append([0, 0, 1])
    iVal += 1

In [19]:
print('shape = ', np.shape(fileTrain))
print('shape index = ', np.shape(indexTrain))

shape =  (1500, 100, 150, 1)
shape index =  (1500, 3)


In [0]:
nb_train_samples = 500*3
nb_validation_samples = 150*3
epochs = 15
batch_size = 32

In [0]:
model = Sequential()
# model.add(Input(shape=(100,150,1)))
model.add(ZeroPadding2D(padding=(2,2), input_shape=(img_height, img_width, 1)))
model.add(Conv2D(32,(5,5),activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4), strides= 2))
model.add(Conv2D(32, (5, 5),activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4), strides= 1))
# model.add(Conv2D(64, (5, 5),activation='relu', strides=1))
# model.add(Conv2D(128, (5, 5),activation='relu', strides=1))
model.add(Flatten())

# model.add(Dense(512))
# model.add(Activation('relu'))
#
model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(3))
model.add(Activation('sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy',
            optimizer='SGD',
            # optimizer='adam',
            metrics=['accuracy'])

In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        # horizontal_flip=True
)
train_datagen.fit(x=fileTrain)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    # horizontal_flip=True
)
validation_datagen.fit(x=fileValidation)

In [0]:
fileTrain = np.array(fileTrain)
indexTrain = np.array(indexTrain)

fileValidation = np.array(fileValidation)
indexValidation = np.array(indexValidation)

In [25]:
print('shape 2 = ', np.shape(fileTrain))
print('shape 2 index = ', np.shape(indexTrain))


shape 2 =  (1500, 100, 150, 1)
shape 2 index =  (1500, 3)


In [29]:
model.fit_generator(
    train_datagen.flow(
        x=fileTrain,
        y=indexTrain,
        batch_size=32,
    ),
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=train_datagen.flow(
        x=fileValidation,
        y=indexValidation,
        batch_size=32,
    ),
    validation_steps=nb_validation_samples // batch_size)

print(model.summary())
# model.save_weights('BismillahFirst-5epochs-W.h5')
model.save('model-dicoding-razif.h5')

Epoch 1/15
46/46 [==============================] - 53s 1s/step - loss: 0.3549 - accuracy: 0.8470 - val_loss: 0.2817 - val_accuracy: 0.8512
Epoch 2/15
46/46 [==============================] - 53s 1s/step - loss: 0.2755 - accuracy: 0.8862 - val_loss: 0.4950 - val_accuracy: 0.8086
Epoch 3/15
46/46 [==============================] - 53s 1s/step - loss: 0.2490 - accuracy: 0.8951 - val_loss: 0.2783 - val_accuracy: 0.9019
Epoch 4/15
46/46 [==============================] - 53s 1s/step - loss: 0.2142 - accuracy: 0.9192 - val_loss: 0.2756 - val_accuracy: 0.8907
Epoch 5/15
46/46 [==============================] - 53s 1s/step - loss: 0.1820 - accuracy: 0.9260 - val_loss: 0.1482 - val_accuracy: 0.9211
Epoch 6/15
46/46 [==============================] - 56s 1s/step - loss: 0.1521 - accuracy: 0.9426 - val_loss: 0.1039 - val_accuracy: 0.9139
Epoch 7/15
46/46 [==============================] - 53s 1s/step - loss: 0.1204 - accuracy: 0.9559 - val_loss: 0.2001 - val_accuracy: 0.9354
Epoch 8/15
46/46 [==